## Part A1: Custom model on CIFAR-10

For A1, we're asked to implement a custom model on CIFAR-10. I'll be using a CNN model and saving it to the ONNX format for portability.

For target accuracy, we're asked to get within "3-5% of industry standard" accuracy. Based on this, we'll target 90% accuracy on the test set.

In [34]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import pandas as pd
import datetime

In [35]:
# Hyperparameters
LEARN_RATE = 0.001
NUM_EPOCHS = 30

In [36]:
# TensorBoard setup
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(f"runs/cifar10_experiment_1_{current_time}")

In [37]:
# Define a dataframe for results
df = pd.read_csv("a1_results.csv")
df.head()

,num_epochs,learn_rate,test_accuracy
0,10,0.0010,0.7015
1,10,0.0001,0.6631
2,30,0.0010,0.7060
3,30,0.0010,0.7111
4,5,0.0010,0.7543


In [38]:
# Make sure we're using GPU
device = torch.device(
    "mps"  # for macOS
    if torch.backends.mps.is_available()
    else "cuda" if torch.cuda.is_available() else "cpu"
)
device

device(type='mps')

### Step 1: Setup and Data Preparation

In [39]:
# We'll use this to normalize the data
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

In [40]:
# Load CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
train_dataset

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [41]:
test_dataset = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)
test_dataset

Files already downloaded and verified


Dataset CIFAR10
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [42]:
# Splitting train dataset into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

In [43]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

### Step 2: Model Design

We will create a simple CNN model.

In [45]:
class EnhancedCNN(nn.Module):
    def __init__(self):
        super(EnhancedCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = EnhancedCNN().to(device)
model

EnhancedCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=8192, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=10, bias=True)
)

### Step 3: Training the Model

In [46]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARN_RATE)

In [47]:
# Training loop
def train_model(num_epochs):
    for epoch in tqdm(range(num_epochs)):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Log loss to TensorBoard
            writer.add_scalar("Loss/train", loss.item(), epoch * len(train_loader) + i)
            
            # # Print train loss
            # if i % 100 == 0:
            #     print(f"Epoch {epoch}, iter {i}: {loss.item()}")
        

    # Log to TensorBoard
    writer.add_graph(model, inputs)


train_model(NUM_EPOCHS)
writer.close()

100%|██████████| 30/30 [24:36<00:00, 49.21s/it]


### Step 4: Evaluation and Saving the Model

In [48]:
def evaluate_model():
    model.eval()  # Set model to evaluation mode
    total = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


results = evaluate_model()
print(f"Accuracy of the model on the validation images: {results * 100}%")

Accuracy of the model on the validation images: 79.88%


In [49]:
# Write results to the dataframe
new_row = pd.DataFrame(
    [[NUM_EPOCHS, LEARN_RATE, results]],
    columns=["num_epochs", "learn_rate", "test_accuracy"],
)
df = pd.concat([df, new_row], ignore_index=True)
df.head()

,num_epochs,learn_rate,test_accuracy
0,10,0.0010,0.7015
1,10,0.0001,0.6631
2,30,0.0010,0.7060
3,30,0.0010,0.7111
4,5,0.0010,0.7543


In [50]:
# Save to CSV
df.to_csv("a1_results.csv", index=False)

In [54]:
# Save the model
torch.save(model.state_dict(), "a1_model.pth")